In this part we will read the reporter information file. So the code starts by reading the first quarter POR file. WHen it reads the next quarter it appends the POR file from the previous quarter. It outputs a summary text which shows all the changes that re taking place in the POR file when going from one quarter to the next quarter.

The Reporter information file will be stored into the ALL quarters folder of the Output folders.

The ignore cols dictionary contains all the columns for which we are not interested to see the changes in the summary file.

In [24]:
#libraries and paths
import os
import zipfile
from datetime import datetime
import pandas as pd
import numpy as np
import re
import time
import json
from pathlib import Path


input_dir = Path(r"C:\Users\32011\Dropbox\Call reports data\Raw Data\Input files")
output_dir =Path(r"C:\Users\32011\Dropbox\Call reports data\Output")
reporter_info_file = output_dir / "Reporter_infor_file.xlsx"

In [25]:
ignore_cols = {
    "Quarter",
    "Last Date/Time Submission Updated On",
    "FDIC Certificate Number",
    "OCC Charter Number",
    "OTS Docket Number",
    "Primary ABA Routing Number",
    "Financial Institution Address",
    "Financial Institution City",
    "Financial Institution State",
    "Financial Institution Zip Code",
    "Financial Institution Name",
}

In [ ]:
# Load or initialize master_df
import sys
log_file = r"C:\Users\32011\Dropbox\Call reports data\Output\reporter_updates_log.txt"
log = open(log_file, "a", encoding = "utf-8")
def log_print(*args, **kwargs):
    """Print to notebook + write to log file (flush immediately)."""
    print(*args, **kwargs)
    print(*args, **kwargs, file=log)
    log.flush()
start_time = time.time()
if reporter_info_file.exists():
    master_df = pd.read_excel(reporter_info_file, dtype=str)
    # Clean column names only if master_df has columns
    if not master_df.empty:
        master_df.columns = master_df.columns.str.strip().str.replace('"', '')
else:
    master_df = pd.DataFrame()   # empty, will be filled with first POR

# Loop through each quarter folder
for quarter_dir in sorted(input_dir.iterdir()):
    if not quarter_dir.is_dir():
        continue

    por_files = list(quarter_dir.glob("*POR*.txt"))
    if not por_files:
        print(f"No POR file found in {quarter_dir.name}")
        continue

    por_file = por_files[0]
    log_print(f"Reading POR file: {por_file}")

    # Read POR file safely
    por_df = pd.read_csv(por_file, sep="\t", dtype=str)
    por_df.columns = por_df.columns.str.strip().str.replace('"', '')

    # Ensure IDRSSD exists
    if "IDRSSD" not in por_df.columns:
        raise ValueError(f"POR file {por_file} does not contain 'IDRSSD' column!")

    por_df["Quarter"] = quarter_dir.name

    # If master_df is empty, initialize it with por_df
    if master_df.empty:
        master_df = por_df.copy()
        master_df.to_excel(reporter_info_file, index=False)
        log_print(f"Reporter file initialized → {reporter_info_file}")
        continue

    # Compare current POR with master_df
    prev_ids = set(master_df["IDRSSD"])
    current_ids = set(por_df["IDRSSD"])

    # Additions
    new_ids = current_ids - prev_ids
    for nid in new_ids:
        log_print(f"+ + + + In Quarter {quarter_dir.name}, new bank added: {nid}")

    # Deletions
    deleted_ids = prev_ids - current_ids
    for did in deleted_ids:
        log_print(f"- - - - In Quarter {quarter_dir.name}, bank removed: {did}")

    # Updates
    common_ids = current_ids & prev_ids
    for cid in common_ids:
        old_row = master_df.loc[master_df["IDRSSD"] == cid].iloc[-1]
        new_row = por_df.loc[por_df["IDRSSD"] == cid].iloc[0]

        changes = []
        for col in por_df.columns:
            if col not in ignore_cols and col in old_row.index:
                if pd.notna(new_row[col]) and new_row[col] != old_row[col]:
                    changes.append((col, old_row[col], new_row[col]))

        if changes:
            log_print(f"In Quarter {quarter_dir.name}, bank {cid} updated:")
            for col, old_val, new_val in changes:
                log_print(f"    - {col}: '{old_val}' → '{new_val}'")

            # Update only existing columns to avoid breaking master_df
            cols_to_update = [col for col in por_df.columns if col in master_df.columns]
            master_df.loc[master_df["IDRSSD"] == cid, cols_to_update] = new_row[cols_to_update].values

    # Add any new banks from this quarter to master_df
    new_rows = por_df[~por_df["IDRSSD"].isin(master_df["IDRSSD"])]
    master_df = pd.concat([master_df, new_rows], ignore_index=True)

    # Save updated master_df
    master_df.to_excel(reporter_info_file, index=False)
    log_print(f"Reporter file updated → {reporter_info_file}")
log.close()
log_print(f"Logs saved to {os.path.abspath(log_file)}")
end_time = time.time()
time_taken = end_time - start_time

Reading POR file: C:\Users\32011\Dropbox\Call reports data\Raw Data\Input files\2001-03-31\FFIEC CDR Call Bulk POR 03312001.txt
Reporter file updated → C:\Users\32011\Dropbox\Call reports data\Output\Reporter_infor_file.xlsx
Reading POR file: C:\Users\32011\Dropbox\Call reports data\Raw Data\Input files\2001-06-30\FFIEC CDR Call Bulk POR 06302001.txt
+ + + + In Quarter 2001-06-30, new bank added: 3028902
+ + + + In Quarter 2001-06-30, new bank added: 2998660
+ + + + In Quarter 2001-06-30, new bank added: 3027174
+ + + + In Quarter 2001-06-30, new bank added: 700672
+ + + + In Quarter 2001-06-30, new bank added: 3017737
+ + + + In Quarter 2001-06-30, new bank added: 3010363
+ + + + In Quarter 2001-06-30, new bank added: 835408
+ + + + In Quarter 2001-06-30, new bank added: 2942823
+ + + + In Quarter 2001-06-30, new bank added: 3009778
+ + + + In Quarter 2001-06-30, new bank added: 2907439
+ + + + In Quarter 2001-06-30, new bank added: 2989006
+ + + + In Quarter 2001-06-30, new bank adde

In [30]:
print(time_taken)


6565.010626316071


In [31]:
input_dir = Path(r"C:\Users\32011\Dropbox\Call reports data\Raw Data\Input files")
output_dir = Path(r"C:\Users\32011\Dropbox\Call reports data\Output\Final Output folder")

OUT_FILE = output_dir / "Master_Dictionary_RC_RI_SU.xlsx"
KEEP_TAGS = {"RC", "RI", "SU"}
ENCODING = "latin-1"

In [38]:
def clean_token(s: str) -> str:
    """Normalize a code/description for safe comparison while keeping original for storage."""
    if s is None:
        return ""
    s = s.strip()
    s = s.replace('"', '')  # remove quotes
    s = re.sub(r'\s+', ' ', s)  # collapse whitespace
    return s


In [39]:
if OUT_FILE.exists():
    df_master = pd.read_excel(OUT_FILE, dtype=str)
    df_master["Code"] = df_master["Code"].astype(str).str.strip().str.replace('"', '')
    df_master["Description"] = df_master["Description"].astype(str).str.strip()
    master_dict = dict(zip(df_master["Code"], df_master["Description"]))
    print(f"Loaded existing master dictionary with {len(master_dict)} codes from {OUT_FILE}")
else:
    master_dict = {}
    print("Starting with empty master dictionary.")

Starting with empty master dictionary.


In [40]:
def read_two_header_lines(file_path: Path):
    """Read first two non-empty lines from file (fast, memory-friendly)."""
    with file_path.open("r", encoding=ENCODING, errors="replace") as f:
        first, second = None, None
        # read up to a few lines until we get two non-empty header lines (skip blank lines)
        for _ in range(20):  # defensive; headers should be near top
            line = f.readline()
            if not line:
                break
            stripped = line.strip("\n\r")
            if stripped.strip() == "":
                continue
            if first is None:
                first = stripped
                continue
            if second is None:
                second = stripped
                break
    return first or "", second or ""

In [43]:
# Flag: are we starting with an empty master? (first processed quarter will initialize)
initial_master = (len(master_dict) == 0)
previous_master_keys = set(master_dict.keys())

# Loop through quarter folders (sorted)
for quarter_dir in sorted([p for p in input_dir.iterdir() if p.is_dir()]):
    quarter_name = quarter_dir.name
    print("\n" + "="*80)
    print(f"Processing quarter folder: {quarter_name}")
    print("="*80)

    quarter_codes = {}  # code -> description (first seen)
    files_processed = 0

    for fpath in sorted(quarter_dir.glob("*.txt")):
        fname = fpath.name
        if not any(tag in fname for tag in KEEP_TAGS):
            continue

        files_processed += 1
        # print(f"  → Reading file: {fname}")

        header1, header2 = read_two_header_lines(fpath)
        codes = [clean_token(x) for x in header1.split("\t")]
        descriptions = [clean_token(x) for x in header2.split("\t")]

        if len(codes) and codes[0].upper().replace('"', '') == "IDRSSD":
            codes = codes[1:]
            if len(descriptions) >= 1:
                descriptions = descriptions[1:]

        if len(descriptions) < len(codes):
            descriptions = descriptions + [""] * (len(codes) - len(descriptions))

        for code, desc in zip(codes, descriptions):
            if code == "":
                continue
            if code not in quarter_codes:
                quarter_codes[code] = desc

    if files_processed == 0:
        print(f"  (No RC/RI/SU schedule files found in {quarter_name})")
        continue

    # --- If this is the very first quarter processed and master was empty, initialize quietly ---
    if initial_master:
        master_dict.update(quarter_codes)      # add all codes quietly
        df_out = pd.DataFrame(list(master_dict.items()), columns=["Code", "Description"])
        df_out.to_excel(OUT_FILE, index=False)
        print(f"Initialized master dictionary from first quarter '{quarter_name}':")
        print(f"  - Files read: {files_processed}")
        print(f"  - Codes added: {len(quarter_codes)} (not listed individually)")
        initial_master = False
        previous_master_keys = set(master_dict.keys())
        continue
    # -------------------------------------------------------------------------------

    # Normal comparison/update flow for subsequent quarters
    quarter_keys = set(quarter_codes.keys())
    prev_keys = set(master_dict.keys())

    added = quarter_keys - prev_keys
    for c in sorted(added):
        master_dict[c] = quarter_codes[c]
        print(f"➕ [{quarter_name}] NEW code added: {c}  →  '{quarter_codes[c]}'")

    removed = prev_keys - quarter_keys
    for c in sorted(removed):
        print(f"➖ [{quarter_name}] Code removed (missing this quarter): {c}  (previous desc: '{master_dict[c]}')")
        master_dict.pop(c, None)

    common = quarter_keys & prev_keys
    for c in sorted(common):
        old_desc = clean_token(master_dict.get(c, ""))
        new_desc = clean_token(quarter_codes[c])
        if new_desc and old_desc != new_desc:
            print(f"🔁 [{quarter_name}] Description changed for {c}:")
            print(f"      '{master_dict.get(c, '')}'  →  '{quarter_codes[c]}'")
            master_dict[c] = quarter_codes[c]

    df_out = pd.DataFrame(list(master_dict.items()), columns=["Code", "Description"])
    df_out.to_excel(OUT_FILE, index=False)
    print(f"Saved master dictionary with {len(master_dict)} codes → {OUT_FILE}")

    previous_master_keys = set(master_dict.keys())

print("\nAll done. Final master dictionary saved at:", OUT_FILE)



Processing quarter folder: 2001-03-31
➕ [2001-03-31] NEW code added: RCFD0073  →  'BAL WITH FGN BRCHS OF OTHER U.S. BKS'
➕ [2001-03-31] NEW code added: RCFD0074  →  'BAL WITH OTHER BKS IN FGN COUNTRIES'
➕ [2001-03-31] NEW code added: RCFD0083  →  'BALS DUE FR US BRNCHS&AGENC OF FGNBK'
➕ [2001-03-31] NEW code added: RCFD0085  →  'BALS DUE FR OTH DEPOSITORY INSTS-US'
➕ [2001-03-31] NEW code added: RCFD0426  →  'OTHER IDENTIFIABLE INTANGIBLE ASSETS'
➕ [2001-03-31] NEW code added: RCFD1257  →  'LSE FNC RECS OF US ADDRS,PAS DU 30'
➕ [2001-03-31] NEW code added: RCFD1258  →  'LSE FIN RECS OF US ADDRS, PAST DUE 9'
➕ [2001-03-31] NEW code added: RCFD1259  →  'LSE FIN RECS OF US ADDR,NONACCRUAL'
➕ [2001-03-31] NEW code added: RCFD1271  →  'LEASE FINAN REC OF NON-US P/D30-89DY'
➕ [2001-03-31] NEW code added: RCFD1272  →  'LSE FIN RECS OF NONUS ADR,PAS DU 90'
➕ [2001-03-31] NEW code added: RCFD1289  →  'US OBLGS ISSD BY US GOVT AGCS-AMRTZ'
➕ [2001-03-31] NEW code added: RCFD1290  →  'US OBLGS IS

In [46]:
!jupyter nbconvert --to script "Call_reports_version2_Part_A.ipynb"


[NbConvertApp] Converting notebook Call_reports_version2_Part_A.ipynb to script
[NbConvertApp] Writing 2047 bytes to Call_reports_version2_Part_A.py


In [51]:
!jupyter nbconvert --to script "Call-reports-Data-Version-2-PART-C.ipynb"

[NbConvertApp] Converting notebook Call-reports-Data-Version-2-PART-C.ipynb to script
[NbConvertApp] Writing 9082 bytes to Call-reports-Data-Version-2-PART-C.py


In [50]:
!jupyter nbconvert --to script "Call-reports-Data-Version-2-PART-D.ipynb"

[NbConvertApp] Converting notebook Call-reports-Data-Version-2-PART-D.ipynb to script
[NbConvertApp] Writing 9392 bytes to Call-reports-Data-Version-2-PART-D.py
